In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import traceback
import threading
from queue import Queue

In [2]:
list_all_url_movie = pd.read_csv('E:/Crawl_Web/Rotten Tomatoes/Crawl URL/URL_movie.csv').squeeze('columns').tolist()

In [3]:
def openMultiChrome(n):
    list_driver = []
    for i in range(n):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument("--start-maximized")
        s = Service('C:\chromedriver')
        driver = webdriver.Chrome(service=s, options=options)
        driver.implicitly_wait(0)

        list_driver.append(driver)
    return list_driver

In [19]:
def getUrl(driver, url):
    driver.get(url)

    # element = WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
    # html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
    # soup = BeautifulSoup(html_of_interest, 'lxml')

    # return soup

In [5]:
def loadMultiUrl(list_driver, list_url):
    num = 0
    for driver in list_driver:
        t = threading.Thread(target=getUrl, args=(driver, list_url[num]))
        t.start()
        num+=1

In [22]:
def getData(driver):
    try:
        element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
        html_of_interest = driver.execute_script('return arguments[0].innerHTML',element)
        soup = BeautifulSoup(html_of_interest, 'lxml')
        
        print("Page is ready!")
        # dict_info_movie = {}

        # # Tiêu đề phim
        # dict_info_movie['Title'] = soup.select('h1[class="scoreboard__title"]')[0].text

        # # Điểm đánh giá của nhà phê bình
        # dict_info_movie['Tomatometer score'] = soup.select('score-board')[0].attrs['tomatometerscore']

        list_title = []
        list_score = []

        list_title.append(soup.select('h1[class="scoreboard__title"]')[0].text)
        list_score.append(soup.select(soup.select('score-board')[0].attrs['tomatometerscore']))
    except Exception:
        traceback.print_exc()
        list_title = []
        list_score = []
    print("Crawl Done!!!")
    return list_title, list_score

In [14]:
def runInParallel(func, list_driver):
    for driver in list_driver:
        que = Queue()
        print("-------Running parallel---------")
        t1 = threading.Thread(target=lambda q, arg1: q.put(func(arg1)), args=(que, driver))
        t1.start()
    try:
        output = que.get()
    except:
        output = []

    return output

In [23]:
num_thread = 2
drivers = openMultiChrome(num_thread)
a_list = []

num = 0
for i in range(round(len(list_all_url_movie)/num_thread)):
    url = list_all_url_movie[num : num+num_thread]
    loadMultiUrl(drivers, url)

    title_score = runInParallel(getData, drivers)
    a_list.append(title_score)

    num += num_thread
    break

-------Running parallel---------
-------Running parallel---------
Crawl Done!!!


Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_10688\857374492.py", line 3, in getData
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
  File "c:\Users\Nguyen Quang Tu\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x00F1DCE3]
	(No symbol) [0x00EB39D1]
	(No symbol) [0x00DC4DA8]
	(No symbol) [0x00DF019F]
	(No symbol) [0x00DF03AB]
	(No symbol) [0x00E1EE62]
	(No symbol) [0x00E0AF14]
	(No symbol) [0x00E1D57C]
	(No symbol) [0x00E0ACC6]
	(No symbol) [0x00DE6F68]
	(No symbol) [0x00DE80CD]
	GetHandleVerifier [0x01193832+2506274]
	GetHandleVerifier [0x011C9794+2727300]
	GetHandleVerifier [0x011CE36C+2746716]
	GetHandleVerifier [0x00FC6690+617600]
	(No symbol) [

Traceback (most recent call last):
  File "C:\Users\Nguyen Quang Tu\AppData\Local\Temp\ipykernel_10688\857374492.py", line 3, in getData
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[id="main_container"]')))
  File "c:\Users\Nguyen Quang Tu\AppData\Local\Programs\Python\Python39\lib\site-packages\selenium\webdriver\support\wait.py", line 95, in until
    raise TimeoutException(message, screen, stacktrace)
selenium.common.exceptions.TimeoutException: Message: 
Stacktrace:
Backtrace:
	(No symbol) [0x00F1DCE3]
	(No symbol) [0x00EB39D1]
	(No symbol) [0x00DC4DA8]
	(No symbol) [0x00DF019F]
	(No symbol) [0x00DF03AB]
	(No symbol) [0x00E1EE62]
	(No symbol) [0x00E0AF14]
	(No symbol) [0x00E1D57C]
	(No symbol) [0x00E0ACC6]
	(No symbol) [0x00DE6F68]
	(No symbol) [0x00DE80CD]
	GetHandleVerifier [0x01193832+2506274]
	GetHandleVerifier [0x011C9794+2727300]
	GetHandleVerifier [0x011CE36C+2746716]
	GetHandleVerifier [0x00FC6690+617600]
	(No symbol) [

Crawl Done!!!


In [16]:
a_list

[([], [])]